# Capstone Project - The Battle of Neighborhoods
## Introduction/Business Problem

### Paris neighbourhoods review to find gaps on Coffee shops opportunities. There is an interest on a new openning of coffee shop, so we will analize the population density agains the existing coffee shops to select the top 3 neighbourhoods for this business investment. The target audience is the group of investors when optimizing lowering risk of investment and maximize benefits.

## Data sources for the analysis

### Wikipedia neighbourhoods information, we get Population & Area(Hec): Quartiers of Paris (https://en.wikipedia.org/wiki/Quarters_of_Paris) 
### GeoData Quariers Paris 
#### Quartiers geo structure: https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=geojson&timezone=Europe/Berlin
#### Quartiers geo locations: https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true
### Foursquare venues information

## Data capture for the analysis

In [1]:
import pandas as pd
import numpy as np
!pip install folium
import folium as fl
from geopy.geocoders import Nominatim
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install wget
import wget
import json

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,

### Data  Capture

In [2]:
tables = pd.read_html("https://en.wikipedia.org/wiki/Quarters_of_Paris")
df = tables[0][1:]
df.columns = tables[0].iloc[0]
df.is_copy = False
df.drop(df.columns[[5]], axis=1, inplace=True)
df.columns = ['Borough','Neighbourhood', 'Population', 'Area', 'Map']
df[0:10]

,Borough,Neighbourhood,Population,Area,Map
1,"1st arrondissement(Called ""du Louvre"")",1st,Saint-Germain-l'Auxerrois,1672,86.9
2,2nd,Les Halles,8984,41.2,NaN
3,3rd,Palais-Royal,3195,27.4,NaN
4,4th,Place-Vendôme,3044,26.9,NaN
5,"2nd arrondissement(Called ""de la Bourse"")",5th,Gaillon,1345,18.8
6,6th,Vivienne,2917,24.4,NaN
7,7th,Mail,5783,27.8,NaN
8,8th,Bonne-Nouvelle,9595,28.2,NaN
9,"3rd arrondissement(Called ""du Temple"")",9th,Arts-et-Métiers,9560,31.8
10,10th,Enfants-Rouges,8562,27.2,NaN


In [3]:
district = ''
for index, row in df.iterrows():
    if (index % 4 == 1): 
        district = df['Borough'][index]
        df['Neighbourhood'][index] = df['Population'][index]
        df['Population'][index] = df['Area'][index]
        df['Area'][index] = df['Map'][index]
    df['Borough'][index] = district
df.is_copy = False
df.drop(df.columns[[4]], axis=1, inplace=True)
df[0:10]

,Borough,Neighbourhood,Population,Area
1,"1st arrondissement(Called ""du Louvre"")",Saint-Germain-l'Auxerrois,1672,86.9
2,"1st arrondissement(Called ""du Louvre"")",Les Halles,8984,41.2
3,"1st arrondissement(Called ""du Louvre"")",Palais-Royal,3195,27.4
4,"1st arrondissement(Called ""du Louvre"")",Place-Vendôme,3044,26.9
5,"2nd arrondissement(Called ""de la Bourse"")",Gaillon,1345,18.8
6,"2nd arrondissement(Called ""de la Bourse"")",Vivienne,2917,24.4
7,"2nd arrondissement(Called ""de la Bourse"")",Mail,5783,27.8
8,"2nd arrondissement(Called ""de la Bourse"")",Bonne-Nouvelle,9595,28.2
9,"3rd arrondissement(Called ""du Temple"")",Arts-et-Métiers,9560,31.8
10,"3rd arrondissement(Called ""du Temple"")",Enfants-Rouges,8562,27.2


Calculate density Population/Area(Hec)

In [4]:
df["Population"] = pd.to_numeric(df["Population"])
df["Area"] = pd.to_numeric(df["Area"])
df['Density'] = df['Population']/df['Area']
df.head()

,Borough,Neighbourhood,Population,Area,Density
1,"1st arrondissement(Called ""du Louvre"")",Saint-Germain-l'Auxerrois,1672,86.9,19.240506
2,"1st arrondissement(Called ""du Louvre"")",Les Halles,8984,41.2,218.058252
3,"1st arrondissement(Called ""du Louvre"")",Palais-Royal,3195,27.4,116.605839
4,"1st arrondissement(Called ""du Louvre"")",Place-Vendôme,3044,26.9,113.159851
5,"2nd arrondissement(Called ""de la Bourse"")",Gaillon,1345,18.8,71.542553


In [5]:
neighborhoods_paris = df
print('Shape After data cleanup:',neighborhoods_paris.shape)

Shape After data cleanup: (80, 5)


### Merge geo data 

In [6]:
from pandas.io.json import json_normalize  
url = 'https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=geojson&timezone=Europe/Berlin'  
wget.download(url, 'paris_quartiers.json')
geo_data = pd.read_json("paris_quartiers.json")
xy_data = json_normalize(data = geo_data['features'])
xy_data.drop(xy_data.columns[[0,1,2,4,6,7,8,9,10,11,12,13]], axis=1, inplace=True)
xy_data.columns = {'cq','lt'}
xy_data['cq'] = pd.to_numeric(xy_data['cq'])
xy_data.sort_values(by = ['cq'], ascending=True, inplace=True)
print(xy_data.shape)
xy_data.head()

(80, 2)


,cq,lt
37,1,"[48.86065013520992, 2.334910329280203]"
79,2,"[48.86228910809422, 2.344898858311021]"
17,3,"[48.86465997810258, 2.336308918965308]"
70,4,"[48.86701859062769, 2.328581664931251]"
69,5,"[48.86930663808872, 2.333431807658117]"


In [7]:
idx=1
neighborhoods = pd.DataFrame(columns = ['Borough', 'Neighbourhood', 'Population', 'Area', 'Density', 'Latitude', 'Longitude'])
for hood, borough, population, area, density in zip(neighborhoods_paris['Neighbourhood'],neighborhoods_paris['Borough'],neighborhoods_paris['Population'],neighborhoods_paris['Area'],neighborhoods_paris['Density']):
    location = xy_data.query('cq == @idx')['lt'].values
    #print(location[0][0],'-',location[0][1])
    neighborhoods.loc[len(neighborhoods)] = [borough, hood, population, area, density, location[0][0], location[0][1]]
    idx=idx+1
print(neighborhoods.shape)
neighborhoods.head()

(80, 7)


,Borough,Neighbourhood,Population,Area,Density,Latitude,Longitude
0,"1st arrondissement(Called ""du Louvre"")",Saint-Germain-l'Auxerrois,1672,86.9,19.240506,48.860650,2.334910
1,"1st arrondissement(Called ""du Louvre"")",Les Halles,8984,41.2,218.058252,48.862289,2.344899
2,"1st arrondissement(Called ""du Louvre"")",Palais-Royal,3195,27.4,116.605839,48.864660,2.336309
3,"1st arrondissement(Called ""du Louvre"")",Place-Vendôme,3044,26.9,113.159851,48.867019,2.328582
4,"2nd arrondissement(Called ""de la Bourse"")",Gaillon,1345,18.8,71.542553,48.869307,2.333432


## Maps data 

In [8]:
address = 'Paris, FR'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Paris are 48.8566101, 2.3514992.


In [9]:
map_data=r'paris_quartiers.json'

# create map of Paris using latitude and longitude values
map_paris = fl.Map(location=[latitude, longitude], zoom_start=13)
fl.Choropleth(geo_data=map_data,
              data=neighborhoods,
              columns=['Neighbourhood', 'Density'],
              key_on='feature.properties.l_qu',
              fill_color='YlOrRd', 
              fill_opacity=0.7, 
              line_opacity=0.2,
              legend_name='Density Rate in Paris'
            ).add_to(map_paris)


In [10]:

# add markers to map
for lat, lng, borough, neighborhood, population, density in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood'], neighborhoods['Population'], neighborhoods['Density']):
    label = '{}, {}, Population:{}, D:{}'.format(neighborhood, borough, population, density)
    label = fl.Popup(label, parse_html=True)
    fl.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

##  Foursquare venues data collection 

In [11]:
# The code was removed by Watson Studio for sharing.

Foursquare Credentails
CLIENT_ID: LCLH3TP21APF0YU5LZE2O3ZRP3GWXCYL5Z5P1UECNVBQJVJZ


In [12]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Explore Neighborhoods in Paris

In [13]:
paris_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

#### Let's check the size of the resulting venues dataframe

In [14]:
print(paris_venues.shape)
paris_venues.head()

(5583, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Saint-Germain-l'Auxerrois,48.86065,2.33491,Musée du Louvre,48.860847,2.336440,Art Museum
1,Saint-Germain-l'Auxerrois,48.86065,2.33491,La Vénus de Milo (Vénus de Milo),48.859943,2.337234,Exhibit
2,Saint-Germain-l'Auxerrois,48.86065,2.33491,Vestige de la Forteresse du Louvre,48.861577,2.333508,Historic Site
3,Saint-Germain-l'Auxerrois,48.86065,2.33491,"Pavillon des Sessions – Arts d'Afrique, d'Asie...",48.860724,2.332121,Art Museum
4,Saint-Germain-l'Auxerrois,48.86065,2.33491,Mona Lisa | La Joconde,48.860139,2.335337,Exhibit


In [15]:
print('There are {} uniques categories.'.format(len(paris_venues['Venue Category'].unique())))

There are 305 uniques categories.
